## Exercise 04 : A/B test

### Import and connection

In [1]:
import pandas as pd
import sqlite3

try:
  connect = sqlite3.connect('../data/checking-logs.sqlite.sqlite_copy')
  print('OK. Connected to sqlite')

except sqlite3.Error as error:
  print('Error with connection:', error)

OK. Connected to sqlite


In [2]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(test.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
             ELSE 'after' END AS time
       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                           ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, connect)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [3]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(control.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
             ELSE 'after'
             END AS time
             FROM control
             LEFT JOIN deadlines ON control.labname=deadlines.labs
             WHERE labname <> 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                           ELSE 'after'
                           END AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(query, connect)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


percent_test = round((test_results.avg_diff[0] - test_results.avg_diff[1]) / test_results.avg_diff[0] * 100, 0)
print(percent_test, '%')

percent_control = round((control_results.avg_diff[0] - control_results.avg_diff[1]) / control_results.avg_diff[0] * 100, 0)
print(percent_control, '%')

### Close the connection

In [4]:
connect.close()
print('Connection to SQLite closed')

Connection to SQLite closed
